Import Libraries for Modelling





In [6]:
%%capture

!pip install creme scikit-learn pandas numpy joblib

Import Data

In [7]:
import pandas as pd

df = pd.read_csv('nakuru_water.csv')

df.head()

,Area,customers,customer_smart,day,time_of_day,weather,Price_of_water,lts_per_day
0,Golf Estate,London flat8,Golf park,tue,Morning,calm,46.928,2.60
1,Shabab,London flat1,London flat8,sat,lunch_hour,sunny/rainny,24.192,3.13
2,Golf Estate,London flat2,London flat4,tue,Morning,rainy,31.248,5.29
3,Olive,London flat1,Golf park,wen,midnight,sunny/windy,36.120,8.03
4,Shabab,London flat3,London flat4,wen,midnight,sunny,35.336,3.84


Split the Training and Testing set

In [8]:
import numpy as np
from sklearn.model_selection import train_test_split

In [9]:
x = df.loc[:,['day', 'Area', 'time_of_day', 'weather']]

x.head(2)

,day,Area,time_of_day,weather
0,tue,Golf Estate,Morning,calm
1,sat,Shabab,lunch_hour,sunny/rainny


In [10]:
x.shape

(9999, 4)

In [11]:
y = df.Price_of_water

In [12]:
y.head(2)

0    46.928
1    24.192
Name: Price_of_water, dtype: float64

In [13]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)


In [14]:
y.shape

(9999,)

Fveature Engineering X and Y

In [15]:
from sklearn.pipeline import make_pipeline 
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder

In [16]:
xcols = [col for col in x]

xcols

['day', 'Area', 'time_of_day', 'weather']

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
pq = make_pipeline(OneHotEncoder(sparse = False, handle_unknown = "ignore"))

In [19]:
Ct = make_column_transformer((pq, xcols))

Model Selection

In [20]:
from sklearn.linear_model import PassiveAggressiveRegressor

model = PassiveAggressiveRegressor()

In [21]:
pipe = make_pipeline(Ct, model)

In [22]:
pipe1 = pipe.fit(x_train, y_train)

In [23]:
pipe1

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline',
                                                  Pipeline(steps=[('onehotencoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse=False))]),
                                                  ['day', 'Area', 'time_of_day',
                                                   'weather'])])),
                ('passiveaggressiveregressor', PassiveAggressiveRegressor())])

In [24]:
#get a sample from the dataset.
samp = x.sample(1, random_state = 130)

samp

,day,Area,time_of_day,weather
4430,fri,Shabab,evening,sunny/windy


In [25]:
pipe1.predict(samp)

array([18.14896025])

In [26]:
# from sklearn.metrics import accuracy_score

# y_pred = pipe1.predict(x_test)

# accuracy_score(y_test, y_pred)

In [27]:
#saving the model
import joblib
joblib.dump(pipe1, "modell")

['modell']

In [28]:
#load our model
# !pip install joblib
Model = joblib.load("modell")

In [29]:
#creating out of sample data
water = {'day' : 'sat', 'time_of_day' : 'lunch', 'Area' : 'london' 	 , 'weather' : 'sunny'}


In [30]:
#convert to pandas DT
test = pd.DataFrame(water, index = [289])

test

,day,time_of_day,Area,weather
289,sat,lunch,london,sunny


In [31]:
#make prediction
Model.predict(test)

array([15.43404446])